# Описание

В данном блокноте я определил "профессию" нейро-сотрудника, его "внутренний мир" и ключевые "инструкции", которым он должен следовать, чтобы обеспечить реализацию системного промпта для него. Разработал структуру базы знаний для нейро-сотрудника, уделив особое внимание качеству и составу данных. Определился с фреймворком для реализации нейро-сотрудника, выбрав между LlamaIndex и LangChain, при этом постарался использовать русскоязычную контурную LLM. Провел трассировку работы нейро-сотрудника, выявив его слабые стороны и возможные признаки галлюцинаций. При обнаружении галлюцинаций принял меры по их устранению, а также предусмотрел потенциальные риски возникновения галлюцинаций в RAG-системе, применив соответствующие техники из пройденных уроков. Внедрил несколько методов для улучшения RAG, учитывая, что нейро-сотрудники уже хорошо известны благодаря статьям на Хабре, и стремился предложить решение, выделяющееся среди других. Также проработал аспекты безопасности нейро-сотрудника, добавив хотя бы базовую фильтрацию запросов.

Вводные данные:
1. Нейро-сотрудник выполняет роль онлайн менеджера-консультанта;
2. База знаний представляет из себя хранилище графов, информация извлекается из документов формата docx;
3. Фреймворк LlamaIndex;
4. Модель saiga_mistral_7b;
5. Трассировщик arize phoenix.

Задачи нейро-сотрудника:
1. Обращаться к базе данных для получения информации.
2. Отвечать точно, основываясь только на данных из источника.
3. Если информация отсутствует, честно говорить, что ответ неизвестен.
4. Избегать излишних эмоций, фраз и догадок.
5. Понимать ключевые запросы клиентов и проверять их по базе графов.

Данные:

Для примера используется 3 документа docx с чёткой структурой заголовками h1 - h2 - h3.

Ссылка на документы:

https://drive.google.com/drive/folders/1O7GrcFClqyFYjEI3s0G0OzD9YaEsz17r?usp=sharing

# Подготовка

In [ ]:
# Установка всех зависимостей одной командой
pip install \
  llama-index-core \
  "arize-phoenix[evals,llama-index]" \
  gcsfs \
  nest-asyncio \
  "openinference-instrumentation-llama-index>=2.0.0" \
  llama-index \
  ipython \
  langchain \
  pypdf \
  langchain_community \
  llama-index-llms-huggingface \
  llama-index-embeddings-huggingface \
  llama-index-embeddings-langchain \
  langchain-huggingface \
  sentencepiece \
  accelerate \
  bitsandbytes \
  peft \
  llama-index-readers-file \
  docx2txt \
  ipykernel \
  llama-index-postprocessor-colbert-rerank

: 

In [ ]:
# Загрузка документов с облака
# import gdown
# gdown.download('https://drive.google.com/uc?export=download&id=1XT5efO3Um2VonKqRm5KTa9PUzm_bw_Ds')

# !unzip -o neuro-employee-v2-archive.zip -d data

In [22]:
# Путь к директории с документами
source_path = '/home/sigma/projects/neuro-assistant-trader-system/data/'

In [ ]:
from huggingface_hub import login
import os
HF_TOKEN = os.getenv("HF_TOKEN")
if not HF_TOKEN:
    raise ValueError("HF_TOKEN environment variable is not set. Please set it in your .env file or export it.")
login(HF_TOKEN, add_to_git_credential=True)

# Загрузка модели

In [5]:
# Импорт класса для чтения файлов из директории
from llama_index.core import SimpleDirectoryReader

# Импорт класса для чтения DOCX файлов
from llama_index.readers.file import DocxReader

# Импорт класса для создания индекса знаний
from llama_index.core import KnowledgeGraphIndex

# Импорт класса для настроек
from llama_index.core import Settings

# Импорт класса для простого хранилища графов
from llama_index.core.graph_stores import SimpleGraphStore

# Импорт класса для контекста хранения
from llama_index.core import StorageContext

# Импорт класса для встраивания моделей Hugging Face
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Импорт класса для языковых моделей Hugging Face
from llama_index.llms.huggingface import HuggingFaceLLM

# Импорт классов для моделей и конфигураций PEFT (Parameter-Efficient Fine-Tuning)
from peft import PeftModel, PeftConfig

# Импорт классов для автоматической загрузки моделей и токенизаторов из Hugging Face
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

# Импорт библиотеки PyTorch для работы с тензорами и нейронными сетями
import torch

# Импорт класса для встраивания моделей Langchain
from llama_index.embeddings.langchain import LangchainEmbedding

# Импорт класса для конфигурации BitsAndBytes
from transformers import BitsAndBytesConfig

# Импорт класса для создания шаблонов промптов
from llama_index.core.prompts import PromptTemplate

# Импорт модуля для получения имени пользователя
import getpass

# Импорт модуля для работы с операционной системой
import os

In [6]:
def messages_to_prompt(messages):
    # Инициализация пустой строки для промпта
    prompt = ""

    # Перебор всех сообщений
    for message in messages:
        # Если роль сообщения 'system', добавляем его в промпт
        if message.role == 'system':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        # Если роль сообщения 'user', добавляем его в промпт
        elif message.role == 'user':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        # Если роль сообщения 'bot', добавляем его в промпт
        elif message.role == 'bot':
            prompt += f"<s>bot\n"

    # Если промпт не начинается с 'system', добавляем его в начало
    if not prompt.startswith("<s>system\n"):
        prompt = "<s>system\n</s>\n" + prompt

    # Добавляем 'bot' в конец промпта
    prompt = prompt + "<s>bot\n"

    # Возвращаем сформированный промпт
    return prompt

def completion_to_prompt(completion):
    # Формируем промпт для завершения, добавляя 'system', 'user' и 'bot'
    return f"<s>system\n</s>\n<s>user\n{completion}</s>\n<s>bot\n"


In [7]:
# Задаем имя модели
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"

In [8]:
# Определяем параметры квантования, иначе модель не выполниться в колабе
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [9]:
# Создание конфига, соответствующего методу PEFT (в нашем случае LoRA)
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загружаем базовую модель, ее имя берем из конфига для LoRA
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,          # идентификатор модели
    quantization_config=quantization_config, # параметры квантования
    torch_dtype=torch.float16,               # тип данных
    device_map="auto"                        # автоматический выбор типа устройства
)

# Загружаем LoRA модель
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)

# Переводим модель в режим инференса
model.eval()

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 2 files: 100%|██████████| 2/2 [18:20<00:00, 550.22s/it] 
2025-11-19 18:06:30,069 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.89s/it]


In [10]:
# Сохраним модель после квантования для последующего использования.

# output_dir = "/home/sigma/projects/neuro-assistant-trader-system/models/saiga_mistral_7b_nf4"
# model.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)

('/home/sigma/projects/neuro-assistant-trader-system/models/saiga_mistral_7b_nf4/tokenizer_config.json',
 '/home/sigma/projects/neuro-assistant-trader-system/models/saiga_mistral_7b_nf4/special_tokens_map.json',
 '/home/sigma/projects/neuro-assistant-trader-system/models/saiga_mistral_7b_nf4/tokenizer.model',
 '/home/sigma/projects/neuro-assistant-trader-system/models/saiga_mistral_7b_nf4/added_tokens.json')

In [11]:
# Загрузить квантованную модель
# model = AutoModelForCausalLM.from_pretrained(output_dir)

# Загрузить токенизатор
# tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [11]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [12]:
llm = HuggingFaceLLM(
    model=model,             # модель
    model_name=MODEL_NAME,   # идентификатор модели
    tokenizer=tokenizer,     # токенизатор
    max_new_tokens=generation_config.max_new_tokens, # параметр необходимо использовать здесь, и не использовать в generate_kwargs, иначе ошибка двойного использования
    model_kwargs={"quantization_config": quantization_config}, # параметры квантования
    generate_kwargs = {   # параметры для инференса
      "bos_token_id": generation_config.bos_token_id, # токен начала последовательности
      "eos_token_id": generation_config.eos_token_id, # токен окончания последовательности
      "pad_token_id": generation_config.pad_token_id, # токен пакетной обработки (указывает, что последовательность ещё не завершена)
      "no_repeat_ngram_size": generation_config.no_repeat_ngram_size,
      "repetition_penalty": generation_config.repetition_penalty,
      "temperature": generation_config.temperature,
      "do_sample": True,
      "top_k": 50,
      "top_p": 0.95
    },
    messages_to_prompt=messages_to_prompt,     # функция для преобразования сообщений к внутреннему формату
    completion_to_prompt=completion_to_prompt, # функции для генерации текста
    device_map="auto",                         # автоматически определять устройство
)

2025-11-19 18:14:35,685 - WARNING - The model `Open-Orca/Mistral-7B-OpenOrca` and tokenizer `IlyaGusev/saiga_mistral_7b` are different, please ensure that they are compatible.


In [13]:
import nest_asyncio
import phoenix as px

from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)

from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations

2025-11-19 18:14:55,050 - INFO - 📋 Ensuring phoenix working directory: /home/sigma/.phoenix
2025-11-19 18:14:55,086 - INFO - Dataset: phoenix_inferences_a4294187-c799-44cb-be7b-38f95e58830e initialized


In [14]:
nest_asyncio.apply()

In [15]:
session = px.launch_app()

2025-11-19 18:15:05,477 - INFO - 📋 Ensuring phoenix working directory: /home/sigma/.phoenix
2025-11-19 18:15:05,586 - INFO - Context impl SQLiteImpl.
2025-11-19 18:15:05,587 - INFO - Will assume transactional DDL.
2025-11-19 18:15:05,620 - INFO - Running upgrade  -> cf03bd6bae1d, init
2025-11-19 18:15:05,967 - INFO - Running upgrade cf03bd6bae1d -> 10460e46d750, datasets
2025-11-19 18:15:05,974 - INFO - Running upgrade 10460e46d750 -> 3be8647b87d8, add token columns to spans table
2025-11-19 18:15:05,977 - INFO - Running upgrade 3be8647b87d8 -> cd164e83824f, users and tokens
2025-11-19 18:15:05,985 - INFO - Running upgrade cd164e83824f -> 4ded9e43755f, create project_session table
2025-11-19 18:15:05,995 - INFO - Running upgrade 4ded9e43755f -> bc8fea3c2bc8, Add prompt tables
2025-11-19 18:15:06,001 - INFO - Running upgrade bc8fea3c2bc8 -> 2f9d1a65945f, Annotation config migrations
/usr/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based inde

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


In [16]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

# Импорт модуля
from llama_index.core.workflow import Workflow

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
)

2025-11-19 18:15:58,000 - INFO - Use pytorch device_name: cuda:0
2025-11-19 18:15:58,001 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


In [18]:
# Настройка ServiceContext (глобальная настройка параметров LLM)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

In [23]:
# Инициализация Docx Reader
parser = DocxReader()
file_extractor = {".docx": parser}
documents = SimpleDirectoryReader(source_path, file_extractor=file_extractor).load_data()

In [25]:
# Проверяем загруженные документы
for i, doc in enumerate(documents):
    print(f"Документ {i+1}:\n\n{doc.get_content()[:256]}\n\n")

Документ 1:

Текущие акции:

Акция: Скидка на первую стирку

Условия акции

Все новые клиенты прачечной "Византия" могут получить скидку в размере 20% на первую стирку.

Период действия

Акция действует постоянно и доступна для всех новых клиентов, которые оформляют пе


Документ 2:

Описание работы менеджера-консультанта

Общие обязанности

Менеджер-консультант прачечной "Византия" является основным контактным лицом для клиентов, обращающихся с вопросами по услугам прачечной. В задачи консультанта входит предоставление точной информац


Документ 3:

Тестовые заказы для прачечной "Византия"

Заказ №1001

Дата заказа: 01 октября 2024

Клиент: Иванов Иван

Тип услуг: Стирка, Глажка

Количество единиц: 10

Срочность: Обычный

Статус: Принят

Комментарий: Не использовать кондиционер для белья.



Заказ №10




In [26]:
# Создаем простое графовое хранилище
graph_store = SimpleGraphStore()

# Устанавливаем информацию о хранилище в StorageContext
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [27]:
# Запускаем генерацию индексов из документа с помощью KnowlegeGraphIndex
indexKG = KnowledgeGraphIndex.from_documents(
    documents=documents,               # данные для построения графов
    max_triplets_per_chunk=10,        # сколько обработывать триплетов связей для каждого блока данных
    show_progress=True,              # показывать процесс выполнения
    include_embeddings=True,         # включение векторных вложений в индекс для расширенной аналитики
    storage_context=storage_context) # куда сохранять результаты

Processing nodes: 100%|██████████| 9/9 [05:13<00:00, 34.88s/it]


In [28]:
# Если необходимо, то можно сохранить индекс

# Определяем путь для сохранения индекса
storage_path = "/home/sigma/projects/neuro-assistant-trader-system/data/index/"

# Сохраняем контекст хранилища, включая все данные индекса (KnowledgeGraphIndex)
storage_context.persist(persist_dir=storage_path)

In [25]:
# Восстанавливаем контекст хранилища из сохранённой директории
# storage_context = StorageContext.from_defaults(persist_dir=storage_path)

# Загружаем KnowledgeGraphIndex с использованием восстановленного контекста
# indexKG = KnowledgeGraphIndex.from_storage_context(storage_context)

In [29]:
query = "Расскажи о текущих акциях"

query_engine = indexKG.as_query_engine(include_text=True, verbose=True)

message_template =f"""<s>system
Ты — нейро-сотрудник прачечной 'Византия'.
Твоя задача — помогать клиентам с вопросами по услугам прачечной, включая стирку, сушку, глажку, доставку, цены и сроки выполнения заказов.
Ты обращаешься к Источнику, содержащему данные об услугах, ценах, акциях, графику доставки, и отвечаешь строго на основе Источника.
Если информация, которую спрашивает клиент, недоступна в Источнике или ты не уверен в ответе, скажи честно: 'я не знаю'. Не придумывай ответы и не гадай.
Отвечай четко и по существу, без эмоций. Твоя цель — предоставить точную информацию, чтобы клиенту не потребовалось задавать дополнительные вопросы.
В разделе 'Текущие акции' указаны актуальные акции.
Если тебя спрашивают о статусе заказа, ты уточняешь номер заказа и говоришь готов он или нет, другие сведения предоставлять ты не можешь.
</s>
<s>user
Вопрос: {query}
Источник:
</s>
"""

In [30]:
response = query_engine.query(message_template)

print()
print('Ответ:')
print(response.response)

2025-11-19 18:26:15,887 - INFO - > Querying with idx: 5c4f0d1c-4e90-4c41-97f4-e638781c5fd1: Работа с клиентами

Консультант обязан следовать строгому регламенту общения ...


Extracted keywords: ['предоставлять', 'заказа', 'указаны', 'акциях', 'другие', 'уточняешь', 'можешь', 'не', 'дополнительные', 'акции', 'информации', 'ты', 'не можешь', 'текущие', 'предоставления', 'вопросы', 'актуальные', 'клиенту', 'говоришь', 'готов', 'сведения', 'номера', 'задавать']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Manager consultant', 'Must inform', 'Client if there is no relevant information in the database')
('Manager consultant', 'Must not', 'Make up answers or guesses')

Ответ:
Текущие акции:

1. Стирка белья - 50% скидка при оплате картой Visa или MasterCard.
2. Доставка прачечной - бесплатно при заказе более 300 рублей.
3. Глажка белья - 10% скидка при оплате картой Visa, MasterCard или Apple Pay.
4. Услуги по стирке и сушке - 10% скидка при оплате картами Visa, MasterCard или Apple Pay.
5. Акция "Бесплатная стирка" - получение б

# Трассировка запросов

# Улучшение вывода

## Переранжирование

Для начала улучшим релевантность результатов поиска и рекомендаций с использованием модели ColBERT для переранжирования.

In [31]:
from llama_index.postprocessor.colbert_rerank import ColbertRerank

colbert_reranker = ColbertRerank(
    top_n=5,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

In [32]:
query_engine = indexKG.as_query_engine(
    include_text=True,
    verbose=True,
    similarity_top_k=10,
    node_postprocessors=[colbert_reranker],
)

In [33]:
# Задаем максимальное количество токенов (512 - стандартный лимит для большинства моделей)
MAX_TOKENS = 512

def truncate_text(text):
    # Токенизируем текст
    tokenized_text = tokenizer(text, truncation=True, max_length=MAX_TOKENS)
    return tokenizer.decode(tokenized_text['input_ids'], skip_special_tokens=True)

# Пример использования для входного запроса
query = "Расскажи о текущих акциях"
query = truncate_text(query)

# Теперь передайте этот сокращенный запрос в query_engine
response = query_engine.query(query)

print()
print('Ответ:')
print(response.response)

2025-11-19 18:37:12,123 - INFO - > Querying with idx: a5ba7d97-bc55-488b-ac4d-ea49a68961f7: Заказ №1004

Дата заказа: 04 октября 2024

Клиент: Васильева Екатерина

Тип у...
2025-11-19 18:37:12,123 - INFO - > Querying with idx: 6b04385f-e938-4afe-9cdf-d0c55daf4348: Тестовые заказы для прачечной "Византия"

Заказ №1001

Дата заказа: 01 октябр...
2025-11-19 18:37:12,124 - INFO - > Querying with idx: b0284d2e-e8b2-4df9-b7b6-7e4914166881: Описание работы менеджера-консультанта

Общие обязанности

Менеджер-консульта...
2025-11-19 18:37:12,124 - INFO - > Querying with idx: 29b99530-24a7-4d55-aebc-78aff465617f: Возможны варианты экспресс-стирки, а также стирка для деликатных тканей.

Суш...


Extracted keywords: ['расскажи', 'текущие', 'акциях', 'акции']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Text', 'Is on', 'Home sigma sigma projects neuro assistant trader system data job-description-of-manager-consultant')
('File_path', 'Is on', 'Home sigma sigma projects neuro assistant trader data job description of manager consultant')
('General obligations', 'Includes', 'Providing information about terms of execution')
('General obligations', 'Includes', 'Providing information about prices')
('Comment', 'Is', 'Возможна доставка по указанному адресно.')
('Urgency', 'Is', 'Срочный (do 24 часов')
('Date', 'Is', '04 октября 2024')
('Date', 'Is', '05 октября 2024')
('Date', 'Is', '01 октября 2024')
('Date', 'Is', '04 октября 204')


/home/sigma/projects/neuro-assistant-trader-system/venv/lib/python3.12/site-packages/llama_index/postprocessor/colbert_rerank/base.py:120: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  node.score = float(score)



Ответ:
Акций нет.


## Ресортировка

Используем ресортировщику контента. (Опыт показал, что использование colbert_reranker + reorder УХУДШАЕТ ответы.)

In [34]:
from llama_index.core.postprocessor import LongContextReorder
reorder = LongContextReorder()

In [35]:
query_engine = indexKG.as_query_engine(
    include_text=True,
    verbose=True,
    similarity_top_k=10,
    node_postprocessors=[reorder],
)

In [36]:
response = query_engine.query(query)

print()
print('Ответ:')
print(response.response)

2025-11-19 18:40:30,532 - INFO - > Querying with idx: a5ba7d97-bc55-488b-ac4d-ea49a68961f7: Заказ №1004

Дата заказа: 04 октября 2024

Клиент: Васильева Екатерина

Тип у...
2025-11-19 18:40:30,533 - INFO - > Querying with idx: 6b04385f-e938-4afe-9cdf-d0c55daf4348: Тестовые заказы для прачечной "Византия"

Заказ №1001

Дата заказа: 01 октябр...
2025-11-19 18:40:30,533 - INFO - > Querying with idx: b0284d2e-e8b2-4df9-b7b6-7e4914166881: Описание работы менеджера-консультанта

Общие обязанности

Менеджер-консульта...
2025-11-19 18:40:30,533 - INFO - > Querying with idx: 29b99530-24a7-4d55-aebc-78aff465617f: Возможны варианты экспресс-стирки, а также стирка для деликатных тканей.

Суш...


Extracted keywords: ['расскажи', 'текущих', 'акции']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Text', 'Is on', 'Home sigma sigma projects neuro assistant trader system data job-description-of-manager-consultant')
('File_path', 'Is on', 'Home sigma sigma projects neuro assistant trader data job description of manager consultant')
('General obligations', 'Includes', 'Providing information about terms of execution')
('General obligations', 'Includes', 'Providing information about prices')
('Comment', 'Is', 'Возможна доставка по указанному адресно.')
('Urgency', 'Is', 'Срочный (do 24 часов')
('Date', 'Is', '04 октября 2024')
('Date', 'Is', '05 октября 2024')
('Date', 'Is', '01 октября 2024')
('Date', 'Is', '04 октября 204')

Ответ:
Акции нет, но есть специальные предложения. Например, стирка одного белья без сушки можно получить за 100 рублей. А если до

# Модерация

## Альтернативная фильтрация через модель OpenAI

In [ ]:
# !pip install openai==0.28.0

In [ ]:
!pip show openai

In [ ]:
import openai

# Запрос ввода ключа от OpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")

# Указываем API ключ
openai.api_key = os.getenv("OPENAI_API_KEY")

# Список категорий, которые надо заблокировать
BLOCKED_CATEGORIES = ["hate", "violence", "self-harm", "sexual"]

In [ ]:
# Функция модерации с учётом настроек категорий
def moderate_request_gpt35(prompt):
    response = openai.Moderation.create(input=prompt)
    results = response["results"][0]

    # Вывод результатов модерации
    print("Результаты модерации:", results)

    # Проверяем только те категории, которые в списке BLOCKED_CATEGORIES
    flagged_categories = [
        category for category, flagged in results["categories"].items()
        if flagged and category in BLOCKED_CATEGORIES
    ]

    if flagged_categories:
        return False, flagged_categories
    return True, None

# Обработка запроса с фильтрацией и проверкой ответа
def process_request_with_moderation(message_template):
    # Модерация запроса
    allowed, categories = moderate_request_gpt35(message_template)
    if not allowed:
        print(f"Запрос заблокирован из-за содержания в категориях: {categories}")
        return "Запрос отклонён модерацией"

    # Если запрос прошёл модерацию, выполняем query
    response = query_engine.query(message_template)

    # Приведение результата запроса к строке перед модерацией
    response_text = str(response)  # Преобразуем ответ в строку

    # Модерация ответа
    allowed_response, response_categories = moderate_request_gpt35(response_text)
    if not allowed_response:
        print(f"Ответ заблокирован из-за содержания в категориях: {response_categories}")
        return "Ответ отклонён модерацией"

    return response_text

In [ ]:
# Тестовый запрос с потенциально недопустимым содержанием
test_message = "I hate people and want to hurt them."

# Вызов функции модерации с тестовым запросом
response = process_request_with_moderation(test_message)

# Вывод результата
print("Ответ на запрос:", response)


In [ ]:
# Тестовый запрос
test_message = "Tell me about the current promotions at the laundromat"
response = process_request_with_moderation(test_message)
print("Ответ на запрос:", response)

## Альтернативная модерация с переиспользованием основной модели

In [37]:
def handle_query_with_moderation(query):
    # Определение шаблона модерации
    moderation_template =f"""<s>system
    Ты — модератор. Твоя задача — проверять запросы на наличие неподобающего контента.
    Запрещенные категории: sexual, hate, harassment, self-harm, threatening, violence.
    Если запрос пользователя содержит контент из этих категорий, возвращай 'BLOCKED'.
    Если запрос не содержит запрещенного контента, возвращай 'ALLOWED'.
    </s>
    <s>user
    Запрос: {query}
    </s>
    """

    # Запрос на модерацию
    moderation_response = query_engine.query(moderation_template)

    print('Ответ модератора:')
    print(moderation_response.response)

    # Если ответ 'ALLOWED', обработать основной запрос
    if moderation_response.response == 'ALLOWED':
        # Определение основного шаблона
        message_template =f"""<s>system
        Ты — нейро-сотрудник прачечной 'Византия'.
        Твоя задача — помогать клиентам с вопросами по услугам прачечной, включая стирку, сушку, глажку, доставку, цены и сроки выполнения заказов.
        Ты обращаешься к Источнику, содержащему данные об услугах, ценах, акциях, графику доставки, и отвечаешь строго на основе Источника.
        Если информация, которую спрашивает клиент, недоступна в Источнике или ты не уверен в ответе, скажи честно: 'я не знаю'. Не придумывай ответы и не гадай.
        Отвечай четко и по существу, без эмоций. Твоя цель — предоставить точную информацию, чтобы клиенту не потребовалось задавать дополнительные вопросы.
        </s>
        <s>user
        Вопрос: {query}
        Источник:
        </s>
        """

        response = query_engine.query(message_template)

        print('Ответ модели:')
        print(response.response)
    else:
        # Если содержимое заблокировано, вернуть сообщение об этом
        print("Запрос заблокирован. Модель не выполнит дальнейший процесс.")


In [38]:
query = "Расскажи о текущих акциях"

In [39]:
handle_query_with_moderation(query)

2025-11-19 18:42:25,095 - INFO - > Querying with idx: 5c4f0d1c-4e90-4c41-97f4-e638781c5fd1: Работа с клиентами

Консультант обязан следовать строгому регламенту общения ...
2025-11-19 18:42:25,095 - INFO - > Querying with idx: c00e84c6-d315-42eb-a264-f90c6fe701c7: Процесс оформления заказов

Менеджер-консультант помогает клиентам оформить з...
2025-11-19 18:42:25,095 - INFO - > Querying with idx: b0284d2e-e8b2-4df9-b7b6-7e4914166881: Описание работы менеджера-консультанта

Общие обязанности

Менеджер-консульта...
2025-11-19 18:42:25,096 - INFO - > Querying with idx: 39c37570-8cdf-4244-a97b-857592b9631b: Период действия

Акция действует до конца следующего месяца.

Примечания

Ски...
2025-11-19 18:42:25,096 - INFO - > Querying with idx: 481cdd4c-6533-4fdf-8096-bac464430727: Период действия

Акция действует до конца месяца.

Примечания

Скидка применя...
2025-11-19 18:42:25,097 - INFO - > Querying with idx: c90e298d-5d6f-42ac-927d-6936da4ccf7b: Текущие акции:

Акция: Скидка на первую сти

Extracted keywords: ['текущих', 'акции']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Manager consultant', 'Must inform', 'Client if there is no relevant information in the database')
('General obligations', 'Includes', 'Providing information about available services')
('Manager consultant', 'Must follow', 'Strict communication rules with clients')
('Manager consultant', 'Must not', 'Make up answers or guesses')
('Akcija', 'Je', 'Kompilksna usluga "vse vkluceno')
('Kliyent', 'Podtverzhdenie', 'Srokov i stoimosti')
('Podtverzhdenie', 'Kliyentu', 'O priemme zakaza')
('Akcija', 'Ukljucuju_osusivanje', 'Glazhkanje')
('Kliyent', 'Ustalenie', 'Tipa uslugi')
('Services', 'Includes', 'Drying')
> Refine context: для подтверждения клиентом. В случае вопросов к...
Ответ модератора:
Текущих акций нет.
Запрос заблокирован. Модель не выполнит дальнейший процесс.


In [40]:
bad_query = "Я ненавижу людей"

In [41]:
handle_query_with_moderation(bad_query)

2025-11-19 18:43:17,629 - INFO - > Querying with idx: c00e84c6-d315-42eb-a264-f90c6fe701c7: Процесс оформления заказов

Менеджер-консультант помогает клиентам оформить з...
2025-11-19 18:43:17,629 - INFO - > Querying with idx: b0284d2e-e8b2-4df9-b7b6-7e4914166881: Описание работы менеджера-консультанта

Общие обязанности

Менеджер-консульта...
2025-11-19 18:43:17,630 - INFO - > Querying with idx: 39c37570-8cdf-4244-a97b-857592b9631b: Период действия

Акция действует до конца следующего месяца.

Примечания

Ски...
2025-11-19 18:43:17,630 - INFO - > Querying with idx: 5c4f0d1c-4e90-4c41-97f4-e638781c5fd1: Работа с клиентами

Консультант обязан следовать строгому регламенту общения ...
2025-11-19 18:43:17,631 - INFO - > Querying with idx: c90e298d-5d6f-42ac-927d-6936da4ccf7b: Текущие акции:

Акция: Скидка на первую стирку

Условия акции

Все новые клие...
2025-11-19 18:43:17,631 - INFO - > Querying with idx: 481cdd4c-6533-4fdf-8096-bac464430727: Период действия

Акция действует до конца м

Extracted keywords: ['людей', 'Я', 'ненавижу']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Manager consultant', 'Must inform', 'Client if there is no relevant information in the database')
('General obligations', 'Includes', 'Providing information about available services')
('Manager consultant', 'Must not', 'Make up answers or guesses')
('Akcija', 'Je', 'Kompilksna usluga "vse vkluceno')
('Kliyent', 'Podtverzhdenie', 'Srokov i stoimosti')
('Podtverzhdenie', 'Kliyentu', 'O priemme zakaza')
('Kliyent', 'Ustalenie', 'Tipa uslugi')
('Services', 'Includes', 'Washing')
('Services', 'Includes', 'Drying')
('Klienci', 'Use', 'Uslugi')
> Refine context: и предлагает альтернативные варианты решения во...
Ответ модератора:
ALLOWED


2025-11-19 18:43:31,969 - INFO - > Querying with idx: 29b99530-24a7-4d55-aebc-78aff465617f: Возможны варианты экспресс-стирки, а также стирка для деликатных тканей.

Суш...
2025-11-19 18:43:31,970 - INFO - > Querying with idx: 5c4f0d1c-4e90-4c41-97f4-e638781c5fd1: Работа с клиентами

Консультант обязан следовать строгому регламенту общения ...
2025-11-19 18:43:31,970 - INFO - > Querying with idx: b0284d2e-e8b2-4df9-b7b6-7e4914166881: Описание работы менеджера-консультанта

Общие обязанности

Менеджер-консульта...
2025-11-19 18:43:31,971 - INFO - > Querying with idx: 6b04385f-e938-4afe-9cdf-d0c55daf4348: Тестовые заказы для прачечной "Византия"

Заказ №1001

Дата заказа: 01 октябр...
2025-11-19 18:43:31,971 - INFO - > Querying with idx: a5ba7d97-bc55-488b-ac4d-ea49a68961f7: Заказ №1004

Дата заказа: 04 октября 2024

Клиент: Васильева Екатерина

Тип у...
2025-11-19 18:43:31,972 - INFO - > Querying with idx: 39c37570-8cdf-4244-a97b-857592b9631b: Период действия

Акция действует до конца с

Extracted keywords: ['клиент', 'сушка', 'глажка', 'доставка', 'эмоции', 'услуги', 'цена', 'заказов', 'цены', 'прачечная', 'дополнительные', 'нейро', 'график', 'акции', 'сотрудник', 'Византия', 'вопросы', 'ответ', 'незнаю', 'информация', 'сроки', 'нейро-сотрудник', 'выполнение', 'источник', 'недоступна', 'данные', 'точная', 'стирка', 'доставки']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('File_path', 'Is in', 'Home sigma sigma projects neuro assistant trader system data job description of manager consultant')
('File_path', 'Is in', 'Home sigma projects neuro assistant trader system data job description of manager consultant docx')
('Text', 'Is on', 'Home sigma sigma projects neuro assistant trader system data job-description-of-manager-consultant')
('File_path', 'Is on', 'Home sigma projects neuro assistant trader system data job description of manger 

# Заключение

В ходе разработки нейро-сотрудника для онлайн-консультирования была создана многофункциональная система, способная эффективно обрабатывать запросы клиентов, извлекать информацию из структурированных данных и предоставлять точные ответы. В проекте были использованы современные инструменты, такие как LlamaIndex и LlamaGuardModeratorPack, что обеспечило высокий уровень фильтрации нежелательных запросов и безопасность взаимодействия с пользователями.

Особое внимание было уделено процессу обработки документов в формате .docx и построению графовых индексов для эффективного извлечения информации. Внедрение различных постобработчиков, таких как ColbertRerank и LongLLMLinguaPostprocessor, помогло улучшить качество ответов, что значительно повышает пользовательский опыт.

Однако следует отметить, что даже для таких моделей, как использованные в проекте saiga_mistral_7b и LlamaGuard-7b, требуется значительное количество CUDA памяти для корректного выполнения задач. Это подчеркивает важность оптимизации использования ресурсов и выбора соответствующего оборудования, особенно при работе с крупными моделями машинного обучения.

Таким образом, проект продемонстрировал возможность создания эффективного и безопасного нейро-сотрудника, способного решать реальные задачи в области онлайн-консультирования, однако требует внимания к аппаратным ресурсам для достижения оптимальной производительности и максимального сокращения галлюцинаций.